# HW №1
## Task 1

In [1]:
%%capture
!pip3 install qiskit

In [2]:
import numpy as np
import qiskit as qk
from sympy import *

In [3]:
with open('./token', 'r') as token_file:
    token = token_file.read()

In [4]:
%%capture
qk.IBMQ.save_account(token, overwrite = True)
qk.IBMQ.load_account()

In [5]:
provider = qk.IBMQ.get_provider(hub = 'ibm-q')

In [6]:
devices = provider.backends(filters=lambda x: (3 <= x.configuration().n_qubits <= 5) and not x.configuration().simulator)

In [7]:
devices

[<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [ ]:
%%capture
real_backend = qk.providers.ibmq.least_busy(devices)
print(real_backend.configuration().n_qubits)

In [8]:
simd_backend = qk.Aer.get_backend('qasm_simulator')

## Task 2

Since we assume the following form of QBit wavefunction:
$\vert \psi \rangle = \sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle$

Actually we measure kinda projection on $Z$ - axis that corresponds $\theta$ angle. Thus $\phi$ angle describes projection onto $X,~Y$ axes. To fix it we can rotate system (or alternavely Vector) in the following way: $X,~Y,~Z \rightarrow Z,~X,~Y$. To do this we should choose axis of rotation, obviously it is normalized vector $(1,~1,~1)$ and angle to rotate $\alpha = 2 \pi / 3$.

To make rotation itself we should compute corresponding axis. We know the form of such rotations (it can be derived from infinitesimal rotations): $\hat{R}_{\vec{n}} (\alpha)= e^{-i \vec{n} \cdot \hat{\vec{\sigma}} \alpha / 2}$.
Since all sigma's satisfy $\hat{\sigma}_i^2 = \hat{I}$ it can be rewritten in the following form: 

$$\hat{R}_{\vec{n}} (\alpha) = I \cos{\frac{\alpha}{2}} - i \vec{n} \cdot \hat{\vec{\sigma}} \sin{\frac{\alpha}{2}}$$

Thus the function to compute such operator is:

In [9]:
px = np.array([[0, 1 ], [1 , 0]], dtype = 'complex128')
py = np.array([[0,-1j], [1j, 0]], dtype = 'complex128')
pz = np.array([[1, 0 ], [0 ,-1]], dtype = 'complex128')
pi = np.array([[1, 0 ], [0 , 1]], dtype = 'complex128')

In [10]:
def rot_mat(vec, angle):
    assert len(vec) == 3
    lvec = np.array(vec, dtype = 'float64')
    lvec = lvec / np.linalg.norm(lvec)
    mat = lvec[0] * px + lvec[1] * py + lvec[2] * pz
    sq_mat = mat @ mat
    assert np.max(np.abs(sq_mat - pi)) < 1e-4
    res = pi * np.cos(angle / 2) - 1j * mat * np.sin(angle / 2)
    sq_res = np.conjugate(res).T @ res
    assert np.max(np.abs(sq_res - pi)) < 1e-4
    return res

We needs not general but specific rotation:

In [11]:
rot = rot_mat([1, 1, 1], 2 * np.pi / 3)

In [12]:
print(rot)

[[ 0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j  0.5+0.5j]]


We need to choose some random (but normalized) initial state:

In [13]:
state = np.random.rand(2) + 1j * np.random.rand(2)
state = state / np.linalg.norm(state)
psi_angle = np.angle(state[0])
state = np.exp(-1j * psi_angle) * state
assert np.imag(state[0]) < 1.e-4

Since here state has form: 
$\vert \psi \rangle = e^{i\psi} \sin \theta \vert 0 \rangle + e^{i\phi} \cos \theta \vert 1 \rangle$
We can rewrite in the following form:
$\vert \psi \rangle = e^{i\psi} (\sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle)$, thus $\varphi = \phi - \psi$.

In [14]:
varphi = np.angle(state[1])
theta  = np.arcsin(state[0])

In [15]:
varphi, theta

(0.25842828874130347, (0.45107135186981967+3.084019376910896e-17j))

Let's compute symbolic result:

In [16]:
tht, phi = symbols('theta, phi', real = True)

In [17]:
vec0 = Matrix([sin(theta), exp(I * phi) * cos(tht)])

In [18]:
rot_op = Matrix([[1-I, -1-I], [1-I, 1+I]]) / 2

In [19]:
vec1 = rot_op @ vec0

In [20]:
ratio = ((conjugate(vec1[0]) * vec1[0]) / (conjugate(vec1[1]) * vec1[1]))

As we can see ratio of probabilities is:
$$\frac{p(\vert 0 \rangle)}{p(\vert 1 \rangle)} = - \frac{e^{i\phi} (\sin{2\theta} \sin{\phi} + 1)}{e^{i\phi} (\sin{2\theta} \sin{\phi} - 1)} = r$$
Thus the output is:
$$\sin{\phi} \sin{2 \theta} = \frac{r - 1}{r + 1}$$

In [21]:
q = qk.QuantumRegister(1)
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
qc.initialize(state, q)
qc.unitary(rot, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd = res.get_counts()

In [22]:
print(r_simd)

{'0': 39513, '1': 26023}


In [23]:
qc.draw()

┌───────────────────────────────────┐┌─────────┐┌─┐
q0_0: ┤ initialize(0.43593,0.87009+0.23j) ├┤ unitary ├┤M├
      └───────────────────────────────────┘└─────────┘└╥┘
c0: 1/═════════════════════════════════════════════════╩═
                                                       0

Let's compute $\sin \phi$:

In [24]:
r = r_simd['0'] / r_simd['1']
sin_phi = (r - 1) / (r + 1) / np.sin(2 * theta)

In [25]:
res_phi = np.arcsin(sin_phi)

In [26]:
res_phi

(0.2654385959513617-1.32478343578087e-17j)

## Task 3

In [27]:
hm = np.array([[1, 1], [1, -1]], dtype = 'complex128') / np.sqrt(2)
sm = np.array([[1, 0], [0, 1j]], dtype = 'complex128')

In [28]:
hadm = Matrix([[1, 1], [1, -1]]) / sqrt(2)

We get three angles $\alpha, \beta, \gamma$. Steps of Euler rotation are:

1. Perform $\hat{R}_z (\gamma)$
2. Perform $\hat{R}_x (\beta )$
3. Perform $\hat{R}_z (\gamma)$

To do so we need to express $\hat{R}_x$ as combination of $\hat{H},\hat{S},\hat{X}$ and $\hat{R}_z$. It can be done in many different ways. But I prefer to decompose it in the following form: $ \hat{R}_x = \hat{H} \hat{R}_z \hat{H}$.


In [29]:
rx = cos(tht / 2) * eye(2) - I * sin(tht / 2) * px
rz = cos(tht / 2) * eye(2) - I * sin(tht / 2) * pz

In [30]:
custom_rx = hadm @ rz @ hadm
rx - custom_rx

Matrix([
[0, 0],
[0, 0]])

Thus we can write our own $\hat{U}_3$ gate:

In [31]:
def custom_rx(qc, qr, alpha):
    qc.h(qr)
    qc.rz(alpha, qr)
    qc.h(qr)

In [32]:
def custom_u3_v0(qc, qr, alpha, beta, gamma):
    qc.rz(beta, qr)
    custom_rx(qc, qr, - np.pi / 2)
    qc.rz(alpha, qr)
    custom_rx(qc, qr,   np.pi / 2)
    qc.rx(gamma, qr)

Let's get Euler's angles from vector:

In [33]:
def rotMatrixFromZX(vz, vx = [1, 0, 0]):
    vz, vx = np.array(vz), np.array(vx)
    nx = vx / np.linalg.norm(vx)
    nz = vz / np.linalg.norm(vz)
    assert(abs(np.dot(nx, nz)) > 1e-10)
    ny = np.cross(nz, nx)
    return np.array([nx, ny, nz])

In [41]:
def _angleFromSinCos(s, c):
    s, c = float(s), float(c)
    if c > 0:
        return np.arctan(s / c)
    elif c < 0:
        return - np.arctan(- s / c)
    else:
        return np.pi * np.sign(s) / 2
    
def rotMatrixToEuler(m):
    cb = m[2, 2]
    if abs(cb) == 1:
        sb = 0
        cc = 1
        sc = 0
        sa = m[1, 0]
        ca = m[0, 0]
    else:
        sb = sqrt(1 - cb**2)
        cc = m[0, 2] / sb
        sc = m[1, 2] / sb
        ca = - m[2, 0] / sb
        sa = m[2, 1] / sb
    return np.array([_angleFromSinCos(sa, ca), _angleFromSinCos(sb, cb), _angleFromSinCos(sc, cc)])

In [42]:
def vecFromAngles(theta_, phi_):
    return np.array([np.sin(theta_) * np.cos(phi_), np.sin(theta_) * np.sin(phi_), np.cos(theta_)])

Let's view on specific vector:

In [43]:
targ_vec = vecFromAngles(np.pi / 3, np.pi / 3)
targ_mat = rotMatrixFromZX(targ_vec)
targ_ans = rotMatrixToEuler(targ_mat)

In [44]:
q = qk.QuantumRegister(1)
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
#to custom angles
qc.rz(targ_ans[1],     q[0])
qc.h(q[0])
qc.rz(- np.pi / 2, q[0])
qc.h(q[0])
qc.rz(targ_ans[0],  q[0])
qc.h(q[0])
qc.rz(  np.pi / 2, q[0])
qc.h(q[0])
qc.rz(targ_ans[2],  q[0])
qc.measure(q, c)
#back transformation
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd_v0 = res.get_counts()

In [45]:
r_simd_v0

{'0': 49267, '1': 16269}

In [46]:
qc.draw()

┌──────────┐┌───┐┌───────────┐┌───┐┌───────────┐┌───┐┌──────────┐┌───┐»
q8_0: ┤ RZ(pi/3) ├┤ H ├┤ RZ(-pi/2) ├┤ H ├┤ RZ(-pi/3) ├┤ H ├┤ RZ(pi/2) ├┤ H ├»
      └──────────┘└───┘└───────────┘└───┘└───────────┘└───┘└──────────┘└───┘»
c2: 1/══════════════════════════════════════════════════════════════════════»
                                                                            »
«      ┌───────────┐┌─┐
«q8_0: ┤ RZ(-pi/2) ├┤M├
«      └───────────┘└╥┘
«c2: 1/══════════════╩═
«                    0

In [47]:
res_theta = np.arctan(np.sqrt(r_simd_v0['0'] / r_simd_v0['1']))
res_theta, res_theta - np.pi / 3

(1.0492261558656577, 0.0020286046690600745)

In [48]:
q = qk.QuantumRegister(1)
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
#to custom angles
qc.rz(targ_ans[1],     q[0])
qc.h(q[0])
qc.rz(- np.pi / 2, q[0])
qc.h(q[0])
qc.rz(targ_ans[0],  q[0])
qc.h(q[0])
qc.rz(  np.pi / 2, q[0])
qc.h(q[0])
qc.rz(targ_ans[2],  q[0])
#as in previous task
rot = rot_mat([1, 1, 1], 2 * np.pi / 3)
qc.unitary(rot, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd_v0 = res.get_counts()

In [49]:
r_simd_v0

{'0': 4438, '1': 61098}

Let's compare phis using indirect method (by comparing of the measured ratio with the theoretical one)

In [50]:
rat = r_simd_v0['0'] / r_simd_v0['1']
comp_rat = - (np.sin(2 * res_theta) * sin(np.pi / 3) + 1.) / (np.sin(2 * res_theta) * sin(np.pi / 3) - 1.)
ratio_of_ratios = rat / comp_rat
ratio_of_ratios

0.0104604865134373

## Task 4

### Sub-Task 1

In [51]:
hadm = Matrix([[1, 1], [1, -1]]) / sqrt(2)
xmat = Matrix([[0, 1], [1, 0]])

In [52]:
res1 = hadm @ xmat @ hadm

In [53]:
res1

Matrix([
[1,  0],
[0, -1]])

In [54]:
zmat = Matrix([[1, 0], [0, -1]])

In [55]:
res1 - zmat

Matrix([
[0, 0],
[0, 0]])

### Sub-Task 2

Let's find Controlled Z matrix by definition: it should apply $\hat Z$ to first qubit if the second one (control) is in $\vert 1 \rangle_2$ state. That gives us the following transition table:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|----------------------------|
|$\vert 00 \rangle$  | $+1 \cdot \vert 00 \rangle$|
|$\vert 10 \rangle$  | $+1 \cdot \vert 10 \rangle$|
|$\vert 01 \rangle$  | $+1 \cdot \vert 01 \rangle$|
|$\vert 11 \rangle$  | $-1 \cdot \vert 11 \rangle$|

Thus it can be represented in form of matrix as:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 &-1
    \end{bmatrix}    $$
    
Or by formula: $\hat{CZ}: \vert ij \rangle \rightarrow (-1)^{i \cdot j} \vert ij \rangle$, obviously it symmetrical for $i,j$ since that $\hat{CZ}$ is equal in both cases.


### Sub-Task 3

First of all we should determine what is Hadamar operator in terms of 2-qubits. Lets's assume that it is applied to the first one, thus transition table is:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|-----------------------------------------------------------------|
|$\vert 0x \rangle$  | $\frac{1}{\sqrt{2}} \cdot (\vert 0x \rangle + \vert 1x \rangle)$|
|$\vert 1x \rangle$  | $\frac{1}{\sqrt{2}} \cdot (\vert 0x \rangle - \vert 1x \rangle)$|

Or it can be written as a tensor product:
$\hat{H}^{(2)}_{1} = \hat{I}^{(1)}_{2} \otimes \hat{H}^{(1)}_{1}$. That gives us the followinfg form:

In [56]:
from sympy.physics.quantum import TensorProduct

In [57]:
hadm_2_1 = TensorProduct(eye(2), hadm)

In [58]:
hadm_2_1

Matrix([
[sqrt(2)/2,  sqrt(2)/2,         0,          0],
[sqrt(2)/2, -sqrt(2)/2,         0,          0],
[        0,          0, sqrt(2)/2,  sqrt(2)/2],
[        0,          0, sqrt(2)/2, -sqrt(2)/2]])

And vice-versa for hadamar gate on the second qubit:

In [59]:
hadm_2_2 = TensorProduct(hadm, eye(2))

In [60]:
hadm_2_2

Matrix([
[sqrt(2)/2,         0,  sqrt(2)/2,          0],
[        0, sqrt(2)/2,          0,  sqrt(2)/2],
[sqrt(2)/2,         0, -sqrt(2)/2,          0],
[        0, sqrt(2)/2,          0, -sqrt(2)/2]])

The definition of controlled not for the second gate: it should invert the second qubit if the first one is in $\vert 1 \rangle$ state.

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|-------------------|
|$\vert 00 \rangle$  | $\vert 00 \rangle$|
|$\vert 10 \rangle$  | $\vert 11 \rangle$|
|$\vert 01 \rangle$  | $\vert 01 \rangle$|
|$\vert 11 \rangle$  | $\vert 10 \rangle$|

That gives us the following form:

In [61]:
cnot_2 = Matrix([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])

In [62]:
cnot_2

Matrix([
[1, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0],
[0, 1, 0, 0]])

Let's writre the right part of circuit:

In [63]:
rc = hadm_2_1 @ hadm_2_2 @ cnot_2 @ hadm_2_1 @ hadm_2_2

In [64]:
rc

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 1],
[0, 0, 1, 0]])

In [65]:
cnot = Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

In [66]:
rc - cnot

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

### Sub-Task 4

We need to find controlled $e^{i\alpha}$ gate. It should work as: multiply both components 
of the second cubit on $e^{i\alpha}$ if the first one is in $\vert 1 \langle$ state. It's diagram:

|Before &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| After &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|--------------------|--------------------------------------|
|$\vert 00 \rangle$  | $1            \cdot \vert 00 \rangle$|
|$\vert 10 \rangle$  | $e^{i \alpha} \cdot \vert 10 \rangle$|
|$\vert 01 \rangle$  | $1            \cdot \vert 01 \rangle$|
|$\vert 11 \rangle$  | $e^{i \alpha} \cdot \vert 11 \rangle$|

Thus the operator's matrix is the following:

$$  \begin{bmatrix}
    1 & 0 & 0 & 0\\
    0 & e^{i\alpha} & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 &e^{i\alpha}
    \end{bmatrix}    $$
    
Obviously it's just another representation of the following tensor product:

$\hat{CExp} = I^{(1)}_{2} \otimes \hat{U}_1 (\alpha)$ and that corresponds just $U_1$ gate on the first qubit (see Sub-Task 3).